In [13]:
from doc_ufcn import models
from doc_ufcn.main import DocUFCN
import cv2
import numpy as np
import matplotlib.pyplot as plt
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from PIL import Image

AttributeError: module 'huggingface_hub.constants' has no attribute 'HF_HOME'

In [ ]:
processor = TrOCRProcessor.from_pretrained('medieval-data/trocr-medieval-latin-caroline/')
model = VisionEncoderDecoderModel.from_pretrained('medieval-data/trocr-medieval-latin-caroline/')

In [6]:
IMAGE_PATH = "e-codices_bbb-0264_084_small.jpg"
image = cv2.cvtColor(cv2.imread(IMAGE_PATH), cv2.COLOR_BGR2RGB)
image

array([[[238, 237, 235],
        [238, 237, 235],
        [238, 237, 235],
        ...,
        [238, 237, 235],
        [238, 237, 235],
        [238, 237, 235]],

       [[238, 237, 235],
        [238, 237, 235],
        [238, 237, 235],
        ...,
        [238, 237, 235],
        [238, 237, 235],
        [238, 237, 235]],

       [[238, 237, 235],
        [238, 237, 235],
        [238, 237, 235],
        ...,
        [238, 237, 235],
        [238, 237, 235],
        [238, 237, 235]],

       ...,

       [[239, 238, 236],
        [238, 237, 235],
        [238, 237, 235],
        ...,
        [238, 237, 235],
        [238, 237, 235],
        [238, 237, 235]],

       [[239, 238, 236],
        [238, 237, 235],
        [238, 237, 235],
        ...,
        [238, 237, 235],
        [238, 237, 235],
        [238, 237, 235]],

       [[239, 238, 236],
        [238, 237, 235],
        [238, 237, 235],
        ...,
        [238, 237, 235],
        [238, 237, 235],
        [238, 237, 235]]

In [7]:
model_path, parameters = models.download_model('doc-ufcn-generic-historical-line')

model = DocUFCN(len(parameters['classes']), parameters['input_size'], 'cpu')
model.load(model_path, parameters['mean'], parameters['std'])

parameters.yml: 100%|██████████| 151/151 [00:00<00:00, 554kB/s]
model.pth: 100%|██████████| 49.2M/49.2M [00:01<00:00, 44.4MB/s]


In [8]:
detected_polygons, probabilities, mask, overlap = model.predict(
    image, raw_output=True, mask_output=True, overlap_output=True
)

In [12]:
# Create a copy of the original image to draw on
image_with_polygons = image.copy()

# Draw each polygon on the copy of the image
for polygon in detected_polygons:
    # Convert polygon points to integers
    pts = np.array(polygon, np.int32)
    pts = pts.reshape((-1, 1, 2))
    
    # Draw the polygon
    cv2.polylines(image_with_polygons, [pts], isClosed=True, color=(255, 0, 0), thickness=2)

# Combine the original image and the image with polygons side by side
combined_image = np.hstack((image, image_with_polygons))

# Display the combined image
plt.figure(figsize=(12, 6))
plt.imshow(combined_image)
plt.axis('off')  # Turn off axis numbers and ticks
plt.show()

ValueError: cannot reshape array of size 1 into shape (1,2)

In [ ]:
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from PIL import Image

In [ ]:
processor = TrOCRProcessor.from_pretrained('medieval-data/trocr-medieval-latin-caroline/')
model = VisionEncoderDecoderModel.from_pretrained('medieval-data/trocr-medieval-latin-caroline/')
pixel_values = processor(images=image, return_tensors="pt").pixel_values

generated_ids = model.generate(pixel_values)
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]